<a href="https://colab.research.google.com/github/Kseniia-Kuzina/BDT-DZ/blob/main/%D0%9A%D0%BE%D0%BD%D1%82%D1%80%D0%BE%D0%BB%D1%8C_%D0%B7%D0%BD%D0%B0%D0%BD%D0%B8%D0%B9_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Задание

# 1.

1) Запустить ячейку ниже, чтобы получить 2 жанра. Для полученных жанров провести все этапы предварительной обработки текста (как в примере), обучить наивный байесовский классификатор, численно оценить его работу.

In [1]:
 import random
 lst = ['Christian', 'Country', 'Pop', 'Rock', 'R&B']
 print('ваши жанры', random.choice(lst), 'и', random.choice(lst))

ваши жанры Rock и Country


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from pandas.core.common import flatten
from google.colab import drive
drive.mount('/content/drive/')
data = pd.read_csv("/content/drive/MyDrive/кз3-2/dataset.csv")
columns = data[['genre', 'lyrics']]
columns = columns[(columns.genre == 'Rock') | (columns.genre == 'Country')]

columns['lowered'] = columns['lyrics'].str.lower()
columns

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,genre,lyrics,lowered
94,Country,Your cheatin heart will make you weep Youll c...,your cheatin heart will make you weep youll c...
95,Country,I was standing by the window On a dark and cl...,i was standing by the window on a dark and cl...
96,Country,Blue moon of Kentucky keep on shinin Shine on...,blue moon of kentucky keep on shinin shine on...
97,Country,Hear that lonesome whippoorwill He sounds too...,hear that lonesome whippoorwill he sounds too...
98,Country,(Hank Cochran) Make the world go away And ge...,(hank cochran) make the world go away and ge...
...,...,...,...
462,Rock,Cant explain all the feelings that youre maki...,cant explain all the feelings that youre maki...
463,Rock,"One foot on the brake and one on the gas, hey...","one foot on the brake and one on the gas, hey..."
464,Rock,Carry on my wayward son Therell be peace when...,carry on my wayward son therell be peace when...
465,Rock,"Ooh yeah Turn it up Come on Im working hard,...","ooh yeah turn it up come on im working hard,..."


*Токенизация*

In [4]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
columns['tokened'] = tokened

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Удаление стоп-слов

In [5]:
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
noise = stopwords.words('english')
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:
    without_stop.append(", ".join(a))
columns['without_stop'] = without_stop

Лемматизация

In [7]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [8]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:
    lemma.append(", ".join(a))
columns['lemmatized'] = lemma

Разделение данных

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.genre, train_size = 0.7)
columns.genre.value_counts()

Rock       95
Country    87
Name: genre, dtype: int64

Векторизация

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

Классификация

In [11]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)
vectorized_x_test = vectorizer.transform(x_test)

Оценка классификации

In [12]:
from sklearn.metrics import *
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

     Country       0.79      0.41      0.54        27
        Rock       0.61      0.89      0.72        28

    accuracy                           0.65        55
   macro avg       0.70      0.65      0.63        55
weighted avg       0.70      0.65      0.63        55



# 2.

2) Найти (нагуглить) по песне каждого из жанров, которые Вам достались, после необходимой обработки их текстов определить жанр обеих песен с помощью обученной в ходе выполнения предыдущего пункта модели.

In [13]:
music = ['I look at you and my blood boils hot, I feel my temperature rise I want it all, give me what you got, there is hunger in your eyes I am getting closer, baby hear me breathe You know the way to give me what I need Just let me love you and you will never leave Feel my heat taking you higher, Burn with me, Heaven is on fire Paint the sky with desire, Angel fly, Heaven is on fire I got a fever raging in my heart, You make me shiver and shake Baby do not stop, take it to the top, Eat it like a piece of cake You are coming closer, I can hear your breath You drive me crazy when you start to tease You could bring a devil to his knees Feel my heat taking you higher, Burn with me, Heaven is on fire Paint the sky with desire, Angel fly, Heaven is on fire Oho Heaven is on fire, oho Heaven is on fire, oho']
#нижний регистр
music_lower = list(map(str.lower, music))
#токенизация
music_tok = [nltk.word_tokenize(i) for i in music_lower]
music_tokened = list(flatten(music_tok))
#удаление стоп-слов
music_withoutstop = list(set(music_tokened) - set(noise))
#лемматизация
music_pro = [lemmatizer.lemmatize(i) for i in music_withoutstop]

#векторизация
vectorized_mus = vectorizer.transform(music)
clf.predict(vectorized_mus)


array(['Rock'], dtype='<U7')

# 3.

3) С помощью набора данных по ссылке аналогичным образом научить модель отличать тексты песен Дэвида Боуи от текстов песен Пола МакКартни
https://www.kaggle.com/italomarcelo/dataset-lyrics-music-mini

In [15]:
import pandas as pd
from pandas.core.common import flatten
from google.colab import drive
drive.mount('/content/drive/')
dat = pd.read_csv("/content/drive/MyDrive/кз3-2/dataset-lyrics-musics-mini.csv")
columns = dat[['cantorNome', 'letra']]
columns = columns[(columns.cantorNome == 'evanescence') | (columns.cantorNome == 'ac-dc')]

columns['lowered'] = columns['letra'].str.lower()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


*Токенизация*

In [16]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
columns['tokened'] = tokened

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Удаление стоп-слов

In [17]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
noise = stopwords.words('english')
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:
    without_stop.append(", ".join(a))
columns['without_stop'] = without_stop

Лемматизация

In [19]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:
    lemma.append(", ".join(a))
columns['lemmatized'] = lemma

Разделение данных

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.cantorNome, train_size = 0.7)
columns.cantorNome.value_counts()

ac-dc          217
evanescence    154
Name: cantorNome, dtype: int64

Векторизация

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

Классификация

In [23]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)
vectorized_x_test = vectorizer.transform(x_test)

Оценка классификации

In [24]:
from sklearn.metrics import *
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

       ac-dc       0.90      0.98      0.94        55
 evanescence       0.98      0.89      0.94        57

    accuracy                           0.94       112
   macro avg       0.94      0.94      0.94       112
weighted avg       0.94      0.94      0.94       112



# Задание
1. Найти (нагуглить) по песне каждого из жанров, которые Вам достались, после необходимой обработки текстов определить жанр обеих песен с помощью обученной  модели в ходе выполнения пункта `1` в Notebook `Контроль знаний 3-2.ipynb`.

Провести Анализ всез жанров песен в тестовом датасете:
- `Christian`,
- `Country`,
- `Hip-Hop`,
- `Pop`,
- `Rock`,
- `R&B`.

2. Научить модель отличать тексты песен двух певцов(выбрать самостоятельно) примере пункта `3` в Notebook `Контроль знаний 3-2.ipynb`.